<a href="https://colab.research.google.com/github/AlkaidCheng/example/blob/master/XGBoostHParamTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install  --upgrade bayesian-optimization
! pip install ray[tune]

Requirement already up-to-date: bayesian-optimization in /usr/local/lib/python3.6/dist-packages (1.1.0)


In [2]:
!wget https://gitlab.cern.ch/clcheng/mlhep-googlesummerofcode/raw/master/Prerequisite/MachineLearning/QIS_EXAM_200Events.npz

--2020-03-08 02:54:15--  https://gitlab.cern.ch/clcheng/mlhep-googlesummerofcode/raw/master/Prerequisite/MachineLearning/QIS_EXAM_200Events.npz
Resolving gitlab.cern.ch (gitlab.cern.ch)... 188.184.116.40, 188.184.30.115, 188.184.104.112, ...
Connecting to gitlab.cern.ch (gitlab.cern.ch)|188.184.116.40|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9140 (8.9K) [application/zip]
Saving to: ‘QIS_EXAM_200Events.npz.2’

QIS_EXAM_200Events. 100%[===================>]   8.93K  --.-KB/s    in 0s      

2020-03-08 02:54:17 (339 MB/s) - ‘QIS_EXAM_200Events.npz.2’ saved [9140/9140]



In [0]:
import numpy as np

In [0]:
def shuffle_zippedarrays(arrays):
  shape = arrays[0].shape
  assert all(shape == arrays[0].shape for arr in arrays)
  index = np.random.permutation(shape[0])
  return [arr[index] for arr in arrays]

def shuffle_data(input, label):
  output = shuffle_zippedarrays([input,label])
  return output[0], output[1]

def load_data(arrays, shuffle = True):
  data = {}
  for key in arrays:
    input, label = np.array([]), np.array([])
    for klabel in arrays[key].item():
      _input = arrays[key].item()[klabel]
      _label = np.full((_input.shape[0],),int(klabel))
      input = np.concatenate((input,_input),axis=0) if input.size else _input
      label = np.concatenate((label,_label),axis=0) if label.size else _label
    if shuffle:
      input, label = shuffle_data(input, label)
    data[key] = {'input': input, 'label': label}
  return data
def load_train_test_input_labels(arrays, shuffle = True):
  data = load_data(arrays, shuffle)
  return data['training_input']['input'], data['test_input']['input'], data['training_input']['label'], data['test_input']['label']

In [0]:
data = np.load('QIS_EXAM_200Events.npz',allow_pickle=True)
train_input, test_input, train_label, test_label = load_train_test_input_labels(data)

In [8]:
import xgboost as xgb
import numpy as np
import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split
from ray.tune.suggest.hyperopt import HyperOptSearch
from ray.tune.schedulers import AsyncHyperBandScheduler
import ray
from hyperopt import hp
from ray import tune


def XGBCallback(env):
    tune.track.log(**dict(env.evaluation_result_list))


def train_breast_cancer(config):
    data, target = sklearn.datasets.load_breast_cancer(return_X_y=True)
    train_x = train_input
    test_x = test_input
    train_y = train_label
    test_y = test_label
    train_set = xgb.DMatrix(train_x, label=train_y)
    test_set = xgb.DMatrix(test_x, label=test_y)
    bst = xgb.train(
        config, train_set, evals=[(test_set, "eval")], callbacks=[XGBCallback])
    preds = bst.predict(test_set)
    pred_labels = np.rint(preds)
    tune.track.log(
        mean_accuracy=sklearn.metrics.accuracy_score(test_y, pred_labels),
        done=True)

ray.shutdown()
ray.init(webui_host='127.0.0.1')

num_threads = 2

space = {
    "verbosity" : 0,
    "max_depth": hp.choice("max_depth", range(3, 26, 1)),
    "num_threads": 4,
    "objective" : "binary:logistic",
    "booster": "gbtree",
    "eval_metric": "logloss",
    "colsample_bytree" : hp.uniform("colsample_bytree",0.5,1),
    "min_child_weight" : hp.quniform("min_child_weight", 1, 11, 1),
    "subsample" : hp.uniform("subsample", 0.5, 1),
    "eta" :  hp.uniform("eta", 0.01, 0.2),
    "tree_method" : 'gpu_hist',
    "gamma" : hp.uniform("gamma", 0, 10),
    "grow_policy": "lossguide"}
algo = HyperOptSearch(
        space,
        max_concurrent=4,
        metric="mean_accuracy",
        mode="min")
from ray.tune.schedulers import ASHAScheduler
! rm -r /xgboost_result
analysis = tune.run(
    train_breast_cancer,
    num_samples=100,
    verbose =  0,
    resources_per_trial={'gpu': 1},
    search_alg = algo,
    local_dir = '/xgboost_result',
    scheduler=AsyncHyperBandScheduler(metric="mean_accuracy", mode="min"))
print("Best config: ", analysis.get_best_config(metric="mean_accuracy"))

2020-03-08 02:58:13,813	WARNING services.py:586 -- setpgrp failed, processes may not be cleaned up properly: [Errno 1] Operation not permitted.
2020-03-08 02:58:13,818	INFO resource_spec.py:212 -- Starting Ray with 6.59 GiB memory available for workers and up to 3.3 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-03-08 02:58:14,122	INFO services.py:1078 -- View the Ray dashboard at 127.0.0.1:8265
2020-03-08 02:58:18,366	INFO function_runner.py:250 -- tune.track signature detected.


(pid=14107) tcmalloc: large alloc 1073741824 bytes == 0x30bd8000 @  0x7f3ba930e887 0x7f3aaa371dff 0x7f3aaa37cfe0 0x7f3aaa37d447 0x7f3aa9e17ecf 0x7f3aaa374289 0x7f3aaa37524d 0x7f3aaa1bf791 0x7f3aaa1c0c95 0x7f3aaa1d3556 0x7f3aaa136aa5 0x7f3ba6d3fdae 0x7f3ba6d3f71f 0x7f3ba6f535c4 0x7f3ba6f53c33 0x5aa6ec 0x50abb3 0x50c5b9 0x508245 0x50a080 0x50aa7d 0x50c5b9 0x508245 0x50a080 0x50aa7d 0x50d390 0x508245 0x50a080 0x50aa7d 0x50d390 0x509d48
(pid=14107) [0]	eval-logloss:0.677586
(pid=14107) [1]	eval-logloss:0.658818
(pid=14107) [2]	eval-logloss:0.645018
(pid=14107) [3]	eval-logloss:0.638047
(pid=14107) [4]	eval-logloss:0.62439
(pid=14107) [5]	eval-logloss:0.615481
(pid=14107) [6]	eval-logloss:0.606897
(pid=14107) [7]	eval-logloss:0.600198
(pid=14107) [8]	eval-logloss:0.598537
(pid=14107) [9]	eval-logloss:0.590687
(pid=14106) [0]	eval-logloss:0.686648
(pid=14106) [1]	eval-logloss:0.65571
(pid=14106) [2]	eval-logloss:0.652602
(pid=14106) [3]	eval-logloss:0.631189
(pid=14106) [4]	eval-logloss:0.61

2020-03-08 02:58:24,228	WARNING worker.py:1058 -- The actor or task with ID ffffffffffffffffe8f4d8080100 is pending and cannot currently be scheduled. It requires {CPU: 1.000000}, {GPU: 1.000000} for execution and {CPU: 1.000000}, {GPU: 1.000000} for placement, but this node only has remaining {GPU: 1.000000}, {node:172.28.0.2: 1.000000}, {CPU: 2.000000}, {memory: 6.591797 GiB}, {object_store_memory: 2.246094 GiB}. In total there are 0 pending tasks and 1 pending actors on this node. This is likely due to all cluster resources being claimed by actors. To resolve the issue, consider creating fewer actors or increase the resources available to this Ray cluster. You can ignore this message if this Ray cluster is expected to auto-scale.


(pid=14175) [0]	eval-logloss:0.676414
(pid=14175) [1]	eval-logloss:0.661563
(pid=14175) [2]	eval-logloss:0.656147
(pid=14175) [3]	eval-logloss:0.636826
(pid=14175) [4]	eval-logloss:0.633279
(pid=14175) [5]	eval-logloss:0.630422
(pid=14175) [6]	eval-logloss:0.620633
(pid=14175) [7]	eval-logloss:0.616597
(pid=14175) [8]	eval-logloss:0.617185
(pid=14175) [9]	eval-logloss:0.610813
(pid=14191) [0]	eval-logloss:0.690594
(pid=14191) [1]	eval-logloss:0.678124
(pid=14191) [2]	eval-logloss:0.674825
(pid=14191) [3]	eval-logloss:0.672541
(pid=14191) [4]	eval-logloss:0.66007
(pid=14191) [5]	eval-logloss:0.660066
(pid=14191) [6]	eval-logloss:0.658189
(pid=14191) [7]	eval-logloss:0.658185
(pid=14191) [8]	eval-logloss:0.65457
(pid=14191) [9]	eval-logloss:0.645911
(pid=14210) [0]	eval-logloss:0.660753
(pid=14210) [1]	eval-logloss:0.643276
(pid=14210) [2]	eval-logloss:0.631332
(pid=14210) [3]	eval-logloss:0.604786
(pid=14210) [4]	eval-logloss:0.596981
(pid=14210) [5]	eval-logloss:0.592495
(pid=14210) [6

2020-03-08 02:59:54,351	WARNING worker.py:1058 -- The actor or task with ID ffffffffffffffffc31026460100 is pending and cannot currently be scheduled. It requires {CPU: 1.000000}, {GPU: 1.000000} for execution and {CPU: 1.000000}, {GPU: 1.000000} for placement, but this node only has remaining {GPU: 1.000000}, {node:172.28.0.2: 1.000000}, {CPU: 2.000000}, {memory: 6.591797 GiB}, {object_store_memory: 2.246094 GiB}. In total there are 0 pending tasks and 1 pending actors on this node. This is likely due to all cluster resources being claimed by actors. To resolve the issue, consider creating fewer actors or increase the resources available to this Ray cluster. You can ignore this message if this Ray cluster is expected to auto-scale.


...

(pid=15364) [0]	eval-logloss:0.668903
(pid=15364) [1]	eval-logloss:0.649648
(pid=15364) [2]	eval-logloss:0.644995
(pid=15364) [3]	eval-logloss:0.626034
(pid=15364) [4]	eval-logloss:0.612635
(pid=15364) [5]	eval-logloss:0.608351
(pid=15364) [6]	eval-logloss:0.598329
(pid=15364) [7]	eval-logloss:0.592802
(pid=15364) [8]	eval-logloss:0.585287
(pid=15364) [9]	eval-logloss:0.580409
(pid=15383) [0]	eval-logloss:0.676915
(pid=15383) [1]	eval-logloss:0.661545
(pid=15383) [2]	eval-logloss:0.651575
(pid=15383) [3]	eval-logloss:0.638827
(pid=15383) [4]	eval-logloss:0.628072
(pid=15383) [5]	eval-logloss:0.623133
(pid=15383) [6]	eval-logloss:0.615102
(pid=15383) [7]	eval-logloss:0.610921
(pid=15383) [8]	eval-logloss:0.605234
(pid=15383) [9]	eval-logloss:0.598625
(pid=15399) [0]	eval-logloss:0.658071
(pid=15399) [1]	eval-logloss:0.634006
(pid=15399) [2]	eval-logloss:0.627375
(pid=15399) [3]	eval-logloss:0.597467
(pid=15399) [4]	eval-logloss:0.580742
(pid=15399) [5]	eval-logloss:0.57298
(pid=15399) [

2020-03-08 03:01:30,114	WARNING import_thread.py:136 -- The actor 'WrappedTrackFunc' has been exported 100 times. It's possible that this warning is accidental, but this may indicate that the same remote function is being defined repeatedly from within many tasks and exported to all of the workers. This can be a performance issue and can be resolved by defining the remote function on the driver instead. See https://github.com/ray-project/ray/issues/6240 for more discussion.


(pid=15783) [9]	eval-logloss:0.567842
(pid=15799) [0]	eval-logloss:0.667532
(pid=15799) [1]	eval-logloss:0.640027
(pid=15799) [2]	eval-logloss:0.629135
(pid=15799) [3]	eval-logloss:0.617145
(pid=15799) [4]	eval-logloss:0.602354
(pid=15799) [5]	eval-logloss:0.598412
(pid=15799) [6]	eval-logloss:0.592764
(pid=15799) [7]	eval-logloss:0.59129
(pid=15799) [8]	eval-logloss:0.590106
(pid=15799) [9]	eval-logloss:0.582892


2020-03-08 03:01:31,969	INFO tune.py:352 -- Returning an analysis object by default. You can call `analysis.trials` to retrieve a list of trials. This message will be removed in future versions of Tune.


Best config:  {'booster': 'gbtree', 'colsample_bytree': 0.9480213147848217, 'eta': 0.09609772163997372, 'eval_metric': 'logloss', 'gamma': 4.379610693019819, 'grow_policy': 'lossguide', 'max_depth': 6, 'min_child_weight': 7.0, 'num_threads': 4, 'objective': 'binary:logistic', 'subsample': 0.5778642303546857, 'tree_method': 'gpu_hist', 'verbosity': 0}


In [11]:
np.random.randint(1,11, size=200)

array([ 9,  9,  9, 10,  6,  6,  2,  9,  3,  4,  6,  1,  9,  6,  9,  3, 10,
        6,  8,  2,  4,  9,  5, 10,  4,  7,  6,  8,  8,  1, 10,  2,  8,  4,
        8,  3,  7,  7,  1,  6,  6,  7,  9,  1, 10,  2,  3, 10,  1,  4,  3,
        9,  4,  2,  3, 10,  6, 10,  2,  4,  8,  3,  3,  3,  9,  5,  1,  8,
        9,  7,  5,  5,  3,  8,  1,  1,  8,  4,  1,  9,  8,  3,  5,  1,  1,
        5,  6,  7,  1,  2,  2,  1,  5,  2,  5,  9,  7,  8,  7,  3,  9,  3,
        2,  2, 10,  3,  7,  3,  9,  3,  5,  2,  4,  6, 10,  4,  6, 10,  7,
       10,  6, 10,  9,  2,  5,  1,  6,  4,  7, 10,  3,  9,  7,  7,  8,  5,
       10,  2,  1,  7,  6,  1,  9,  7,  9, 10,  7, 10,  1,  9,  7,  5,  8,
        9,  2,  8,  5,  1,  3,  9,  3,  6,  3,  5,  7,  5,  9,  6,  1, 10,
        6,  4,  5, 10,  5,  5, 10,  4, 10,  3,  8,  7,  8,  2,  6,  9,  2,
        3,  8,  4,  9,  3,  8,  3,  7, 10,  6, 10,  5,  1])